In [16]:
import neptune.new as neptune
from utils import *
import numpy as np

In [17]:
project = neptune.get_project(
    name='rahatzamancse/MPSE-mds', 
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3NDk5MWVkNS0wMDg4LTRmNTktYWQyZC0zNzUyNTg0MTY1MGYifQ=='
)

# Get dashboard with runs contributed by 'sophia' tagged 'cycleLR'
run_table_df = project.fetch_runs_table(
    owner='rahatzamancse',
).to_pandas()

run_table_df.columns

Index(['sys/creation_time', 'sys/description', 'sys/failed', 'sys/hostname',
       'sys/id', 'sys/modification_time', 'sys/monitoring_time', 'sys/name',
       'sys/owner', 'sys/ping_time', 'sys/running_time', 'sys/size',
       'sys/state', 'sys/tags', 'GT/Points per perspective list',
       'Results/Alignment/4PointSample/error',
       'Results/Alignment/4Point_ICP/error',
       'Results/Alignment/Global-RMSE-opt/error',
       'Results/Alignment/GlobalRMSE_ICP/error', 'Results/Final Cost',
       'Results/Final Individual Cost',
       'Results/computation history/Actual Iterations Run',
       'Results/computation history/Conclusion',
       'Results/computation history/costs',
       'Results/computation history/grads', 'Results/computation history/lrs',
       'Results/computation history/steps', 'parameters/DATASET',
       'parameters/INITIAL_EMBEDDING', 'parameters/MPSE/BATCH_SIZE',
       'parameters/MPSE/INITIAL_PROJECTIONS', 'parameters/MPSE/MAX_ITER',
       'parameter

In [21]:
run_id = "MPSEMDS-1614"

run = neptune.init(
    project='rahatzamancse/MPSE-mds',
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3NDk5MWVkNS0wMDg4LTRmNTktYWQyZC0zNzUyNTg0MTY1MGYifQ==',
    run=run_id, mode="read-only"
)

print("Ground Truth")
run["GT/X"].download('/tmp/')
points = np.load('/tmp/X.npy')
print(points.min(), points.max())
plot_3D([points]).show()

print("Embedding")
run["Results/Embedding_X"].download('/tmp/')
embedding = np.load('/tmp/Embedding_X.npy')
plot_3D([embedding]).show()

all_trans_mats = []
for metric in ['4PointSample', '4Point_ICP', 'Global-RMSE-opt', 'GlobalRMSE_ICP']:
    path = "Results/Alignment/" + metric + '/transformation_mat'
    run[path].download(f'/tmp/')
    trans_mats = np.load('/tmp/transformation_mat.npy')
    all_trans_mats.append(trans_mats)

# 4Point
print("4PointSample :", run['Results/Alignment/4PointSample/error'].fetch())
aligned_embedding = apply_transformation(embedding, all_trans_mats[0])
plot_3D([points, aligned_embedding]).show()

# 4Point_ICP
print("4PointSample_ICP :", run['Results/Alignment/4Point_ICP/error'].fetch())
aligned_embedding = apply_transformation(apply_transformation(embedding, all_trans_mats[0]), all_trans_mats[1])
plot_3D([points, aligned_embedding]).show()

# Global
print("Global-RMSE-opt :", run['Results/Alignment/Global-RMSE-opt/error'].fetch())
aligned_embedding = apply_transformation(embedding, all_trans_mats[2])
plot_3D([points, aligned_embedding]).show()

# Global_ICP
print("GlobalRMSE_ICP :", run['Results/Alignment/GlobalRMSE_ICP/error'].fetch())
aligned_embedding = apply_transformation(apply_transformation(embedding, all_trans_mats[2]), all_trans_mats[3])
plot_3D([points, aligned_embedding]).show()

print("Projections")
run["GT/Projections"].download('/tmp/')
projections = np.load('/tmp/Projections.npy')
print(projections.min(), projections.max())

run.stop()

https://app.neptune.ai/rahatzamancse/MPSE-mds/e/MPSEMDS-1614
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Ground Truth
-20.188453126978562 21.072405498365484


Embedding


4PointSample : 8.742929634578308


4PointSample_ICP : 9.002057306580905


Global-RMSE-opt : 8.202070540955381


GlobalRMSE_ICP : 22.028998770107737


Projections
-0.9902976003378858 1.0
Shutting down background jobs, please wait a moment...
Done!


In [23]:
min_dist, best_t = get_4pointsample_transform_mat(embedding, points, dist_agg_fn=None, t_sample=10000000)

aligned_points = apply_transformation(embedding, best_t)

plot_3D([points, aligned_points])

100%|█████████████████████████████████| 1000000/1000000 [02:52<00:00, 5792.80it/s]


# For each projections applied to ground truth

In [10]:

for i in range(len(projections)):
    tranformed_points = apply_transformation(points, projections[i])
    tranformed_points[:, 2] = 0

    plot_3D([tranformed_points]).show()